# 0. Sample

In [ ]:
from ase.visualize import view

from src.sample import sample

In [ ]:
help(sample)

In [ ]:
# Sample with Default model trained with alex-mp-20
gen_atoms_list = sample(
    num_samples=1000,
    batch_size=500,
    output_dir="outputs/alex-mp-20",
)

In [ ]:
view(gen_atoms_list[0], viewer="ngl")

In [ ]:
# Sample with mp-20 model
from src.utils.checkpoint import get_checkpoint

vae_ckpt_path = get_checkpoint("mp_20_vae")
ldm_ckpt_path = get_checkpoint("mp_20_ldm")

gen_structures_mp20 = sample(
    num_samples=1000,
    batch_size=500,
    ldm_ckpt_path=ldm_ckpt_path,
    vae_ckpt_path=vae_ckpt_path,
    output_dir="outputs/mp-20",
)

# 1. Evaluate

> **Note**: Before running metrics for evaluation, see [Prerequisites in Evaluation Guide](docs/EVALUATION.md#prerequisites) for setup instructions (e.g., downloading reference datasets, pre-computed features).

In [ ]:
from monty.serialization import loadfn

from src.utils.metrics import Metrics

In [ ]:
help(Metrics)

In [ ]:
# Load generated structures from [0.Sample] step
gen_structures = loadfn("outputs/alex-mp-20/generated_structures.json.gz")

# Initialize metrics calculator
metrics = Metrics(
    metrics=["unique", "novel", "e_above_hull"],
    reference_dataset="mp-20",
    phase_diagram="mp-all",
    metastable_threshold=0.1,
)

In [ ]:
results = metrics.compute(gen_structures)

In [ ]:
# Calculate mSUN score (percentage that are unique, novel, AND metastable)
msun_score = (
    results["unique"] & results["novel"] & results["is_metastable"]
).mean() * 100
print(f"mSUN Score: {msun_score:.2f}%")
print(f"Uniqueness: {results['unique'].mean():.2%}")
print(f"Novelty: {results['novel'].mean():.2%}")
print(f"Metastable: {results['is_metastable'].mean():.2%}")

# Convert to DataFrame for analysis
df = metrics.to_dataframe()
df.head()

# Save results to CSV
df.to_csv("outputs/alex-mp-20/metrics_results.csv")